In [120]:
import re
import csv
import networkx as nx
import matplotlib.pyplot as plt
import os

files = os.listdir("Data/GoT_dyn_ts10")

def get_active_nodes(G):
    activenodes = set()
    for edge in G.edges(data=True):
        #print(edge)
        activenodes.add(edge[0])
        activenodes.add(edge[1])
    return activenodes

def episode(s, e, name = 'GoT_S0'):
    if e < 10:
        stt = name +str(s)+'E0'+str(e) 
    else:
        stt = name +str(s)+'E'+str(e) 
    #print(stt)
    edic = {}
    for f in files:
        if f.startswith(stt):
            #en = int(f[11:14])
            en = int(f.split('.')[0].split('_')[-1])
            #print(f, en)
            edic[en] = f
    return edic
 
def readG(f, path = 'Data/GoT_dyn_ts10/'):
    #print(f)
    return nx.read_graphml( path + f)

def histogram(G):
    hist = {}
    dict = {}
    for node in G.nodes(data=True):
        dict[node[0]] = node[1]['label']
        
    #print(dict)
    for edge in G.edges(data=True):
        for i in range(2):
            node = dict[edge[i]]
            #print(node)
            try:
                hist[node] += 1
            except:
                hist[node] = 1
    return hist

def aggregate_density(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    edic = episode(s,e, name)
    sorted_x = sorted(edic.items(), key=lambda kv: kv[1])
   
    total_edges = 0
    potential_edges = 0
    
    #print(sorted_x)
    for k, v in sorted_x:
        g = readG(v, path)
        
        num_actnodes = len(get_active_nodes(g))
        num_edges = len(g.edges(data=True))
        
        total_edges += num_edges
        potential_edges += num_actnodes*(num_actnodes-1)/2
       
    return total_edges/potential_edges

# TODO: remove the nodes with small strength, re-calculate density again
# density = sum(edge weights) / (num_10s * active_nodes * (active_nodes - 1) / 2)
def weighted_density(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    edic = episode(s,e, name)
    sorted_x = sorted(edic.items(), key=lambda kv: kv[1])
   
    total_edges = 0
    num_episodes = len(sorted_x)
    potential_edges = 0
    
    active_nodes = set()
    
    #print(sorted_x)
    for k, v in sorted_x:
        g = readG(v, path)
        
        node_in_one_episode = get_active_nodes(g)
        active_nodes = active_nodes.union(node_in_one_episode)
        num_edges = len(g.edges(data=True))
        total_edges += num_edges
    
    #print(len(active_nodes))
    num_actnodes = len(active_nodes)
    potential_edges = num_episodes * num_actnodes*(num_actnodes-1)/2
       
    return total_edges/potential_edges

def average_density(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    edic = episode(s,e,name)
    sorted_x = sorted(edic.items(), key=lambda kv: kv[1])
    #print(sorted_x)
    dns = []
    #print(sorted_x)
    for k, v in sorted_x:
        g = readG(v, path)
        #h = histogram(g)
        #print(v)
        dns.append(nx.density(g))
    return sum(dns)/len(dns)
    
def node_dict(g):
    dict = {}
    for node in g.nodes(data=True):
        dict[node[0]] = node[1]['label']
    return dict
            
def active_nodes(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    edic = episode(s,e, name)
    sorted_x = sorted(edic.items(), key=lambda kv: kv[1])
    
    act_nodes = set()
    #print(sorted_x)
    for k, v in sorted_x:
        g = readG(v, path)
        activenodes = get_active_nodes(g)
        nodedict = node_dict(g)
        for n in activenodes:
            act_nodes.add(nodedict[n])
    #print(act_nodes)
    return act_nodes
     
def degree_data(g):
    G_deg = nx.degree_histogram(g)
    G_deg_sum = [a * b for a, b in zip(G_deg, range(0, len(G_deg)))]
    avg_degree = sum(G_deg_sum) / g.number_of_nodes()
    return avg_degree
    
def average_degree(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    edic = episode(s,e,name)
    sorted_x = sorted(edic.items(), key=lambda kv: kv[1])
    
    degrees = []
    act_nodes = set()
    #print(sorted_x)
    for k, v in sorted_x:
        g = readG(v, path)
        avg_degree = degree_data(g)
        degrees.append(avg_degree)
        
    #print(degrees)
    return sum(degrees)/len(degrees)
    
def get_corr(review):
    dict = {'Network Metrics':[],
            'Correlation':[],
            'pValue':[]
           }

    for col in ['weighted_density1', 'weighted_density2', 'active_nodes', 'agg_actnodes', 'avg_degrees', 'avg_strength']:
        corr, pval = pearsonr(review[col], review['Review'])
        dict['Network Metrics'].append(col)
        dict['Correlation'].append(corr)
        dict['pValue'].append(pval)

    df_corr = pd.DataFrame(dict)
    return df_corr


def aggregate_graph(s, e, name , path ):
    edic = episode(s,e,name)
    sorted_x = sorted(edic.items(), key=lambda kv: kv[1])
    
    degrees = []
    count = len(sorted_x)
    act_nodes = set()
    
    newg = {}
    #print(sorted_x)
    max_weight = 0
    for k, v in sorted_x:
        g = readG(v, path)
    
        nd = node_dict(g)
        for node1, node2, data in g.edges(data=True):
            node1  = nd[node1]
            node2  = nd[node2]
            weight = data['weight']
            
            try:
                newg[(node1, node2)] += weight
            except:
                newg[(node1, node2)]  = weight
            
            max_weight = max(max_weight, newg[(node1, node2)])
    
    #scale weights
    for k in newg:
        newg[k] = newg[k] / count
        
    return newg, count, max_weight/count
        

#print(episode_active_nodes(1,1))

def average_strength(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    g, count, max_weight = aggregate_graph(s, e, name, path)
    
    strength = {}
    sum_weight = 0
    sum_degree = 0
    node_list = set()

    for k in g:
        node1, node2 = k
        v = g[k]
        
        #remove small weight edges 
        if v <  max_weight * 0.05:
            continue
        
        sum_weight += v
        sum_degree += 2
        node_list.add(node1)
        node_list.add(node2)
        
    
    cnt_nodes = len(node_list)
    return sum_weight/cnt_nodes,  sum_degree/cnt_nodes, cnt_nodes

# Calculate density using 
# Calculate total weights
# density = sum(edge weights) / (num_10s * active_nodes * (active_nodes - 1) / 2)
def weighted_density1(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    edic = episode(s,e, name)
    sorted_x = sorted(edic.items(), key=lambda kv: kv[1])
   
    total_weight = 0
    potential_edges = 0
    
    
    #print(sorted_x)
    for k, v in sorted_x:
        # get a 10s graph
        g = readG(v, path)
        nd = node_dict(g)
        for node1, node2, data in g.edges(data=True):
            node1  = nd[node1]
            node2  = nd[node2]
            weight = data['weight']
            total_weight += weight
   
            node_in_one_episode = get_active_nodes(g)
            cnt = len(node_in_one_episode)
            potential_edges += cnt * (cnt - 1) /2
            
    return total_weight/potential_edges


# Calculate density using Aggregated Graph per Episode
# 1. Aggregate all 10s graphs in one episode to a single graph 
# 2. Remove edges with less than 5% of max weight 
# 3. Calculate desnity using total edge weight / (n(n-1)/2)
def weighted_density2(s, e, name = 'GoT_S0', path = 'Data/GoT_dyn_ts10/'):
    g, count, max_weight = aggregate_graph(s, e, name, path)
    
    strength = {}
    sum_weight = 0
    node_list = set()

    for k in g:
        node1, node2 = k
        v = g[k]
        
        #remove small weight edges 
        if v <  max_weight * 0.05:
            continue
        
        sum_weight += v
        node_list.add(node1)
        node_list.add(node2)
        
    count = len(node_list)
    #print(sum_weight, len(node_list))
    return sum_weight * 2/ count/(count-1)        

#for i in range(1,10):
#    print(weighted_density1(1,i))    
#print(aggregate_dens)
#print(num_actnodes)
#print(average_degrees)

In [152]:
def get_df(review = 'GoT.csv', path = "Data/GoT_dyn_ts10/", name = 'GoT_S0', lastS = 4, lastE = 11, size = 22):
    weighted_dens1 = []
    weighted_dens2 = []
    num_actnodes = []
    agg_actnodes = []
    average_degrees = []
    average_strength_list = []


    files = os.listdir(path)
    for S in range(1,lastS):
        for E in range(1,lastE):
            #print(S,E)
            if review == 'GoT.csv':
                if S == 3 and E > 2:
                    break
            try:
                sth, dgr, cnt_active_nodes = average_strength(S, E,  name, path)
                average_strength_list.append(sth)
                agg_actnodes.append(cnt_active_nodes)

                weighted_dens1.append(weighted_density1(S, E, name, path))
                weighted_dens2.append(weighted_density2(S, E, name, path))

                num_actnodes.append(len(active_nodes(S, E, name, path)))
                average_degrees.append(dgr)

                episode_nodes.append(epi_nodes)
            except:
                pass

    GoT_review = pd.read_csv(review).drop('Unnamed: 0', 1)  
    review = GoT_review.head(size)

    review2 = review.assign(weighted_density1= weighted_dens1)
    review2 = review2.assign(weighted_density2= weighted_dens2)

    review2 = review2.assign(active_nodes = num_actnodes)
    review2 = review2.assign(agg_actnodes = agg_actnodes)

    review2 = review2.assign(avg_degrees = average_degrees)
    review2 = review2.assign(avg_strength = average_strength_list)

    return review2

#df = get_df()
#df

# Game of Throne

In [153]:
files = os.listdir("Data/GoT_dyn_ts10/")
df_GoT = get_df('GoT.csv',   "Data/GoT_dyn_ts10/",   'GoT_S0', 4, 11,22)
df_GoT

,Episode,Review,weighted_density1,weighted_density2,active_nodes,agg_actnodes,avg_degrees,avg_strength
0,Winter Is Coming,9.001235,0.157542,0.879977,30,26,2.846154,10.999712
1,The Kingsroad,8.701235,0.210869,1.279233,27,24,2.583333,14.711177
2,Lord Snow,8.601235,0.144238,0.780416,46,38,2.684211,14.437690
3,"Cripples, Bastards, and Broken Things",8.701235,0.167519,0.955038,48,34,1.941176,15.758127
4,The Wolf and the Lion,9.001235,0.153667,1.358700,40,30,2.133333,19.701143
5,A Golden Crown,9.101235,0.121972,1.335685,42,32,2.187500,20.703125
6,You Win or You Die,9.201235,0.258854,1.489819,38,28,2.000000,20.112551
7,The Pointy End,9.001235,0.075544,0.469583,53,41,1.951220,9.391670
8,Baelor,9.601235,0.089010,1.361319,50,30,1.733333,19.739130
9,Fire and Blood,9.501235,0.069047,0.529246,49,40,1.650000,10.320304


In [154]:
get_corr(df_GoT)

,Network Metrics,Correlation,pValue
0,weighted_density1,0.218879,0.327757
1,weighted_density2,0.228016,0.307451
2,active_nodes,-0.400721,0.064579
3,agg_actnodes,-0.458978,0.031661
4,avg_degrees,-0.167934,0.455045
5,avg_strength,-0.120952,0.591839


# House of Cards

In [155]:
files = os.listdir("Data/HoC_dyn_ts10")
df_hoc = get_df('HoC.csv', "Data/HoC_dyn_ts10/",  'HoC_S0', 3, 14, 26)
df_hoc

,Episode,Review,weighted_density1,weighted_density2,active_nodes,agg_actnodes,avg_degrees,avg_strength
0,Chapter 1,8.601235,0.483989,1.207103,30,24,2.083333,13.881687
1,Chapter 2,8.501235,0.240416,1.032492,33,26,2.153846,12.906154
2,Chapter 3,8.301235,0.320730,1.574021,33,22,2.000000,16.527222
3,Chapter 4,8.201235,0.441909,0.981159,31,27,2.074074,12.755071
4,Chapter 5,8.401235,0.173820,0.792796,36,29,2.275862,11.099147
5,Chapter 6,8.501235,0.332053,1.564426,33,22,2.272727,16.426470
6,Chapter 7,8.101235,0.275441,1.056570,34,28,2.428571,14.263700
7,Chapter 8,7.701235,0.452259,3.157255,19,15,2.000000,22.100785
8,Chapter 9,8.501235,0.368298,1.837056,23,22,2.545455,19.289084
9,Chapter 10,8.701235,0.818472,1.795357,21,20,2.500000,17.055894


In [156]:
get_corr(df_hoc)

,Network Metrics,Correlation,pValue
0,weighted_density1,0.147383,0.472457
1,weighted_density2,0.089816,0.662594
2,active_nodes,-0.158587,0.439059
3,agg_actnodes,-0.142668,0.486893
4,avg_degrees,-0.087954,0.669197
5,avg_strength,0.107916,0.599765


# Breaking Bad

In [158]:
files = os.listdir("Data/BB_dyn_ts10")
df_bb = get_df('BB.csv', "Data/BB_dyn_ts10/",  'BB_S0', 4, 14, 26)
df_bb

,Episode,Review,weighted_density1,weighted_density2,active_nodes,agg_actnodes,avg_degrees,avg_strength
0,Pilot,9.001235,0.620860,7.243874,18,12,2.000000,39.841304
1,Cat's in the Bag...,8.601235,5.847029,81.456667,8,4,2.000000,122.185000
2,...And the Bag's in the River,8.701235,1.161629,14.267550,14,11,2.000000,71.337752
3,Cancer Man,8.201235,0.486241,2.123505,24,22,2.727273,22.296802
4,Gray Matter,8.301235,0.638527,4.625012,24,15,2.266667,32.375083
5,Crazy Handful of Nothin',9.301235,0.475483,5.157157,23,14,2.142857,33.521519
6,A No-Rough-Stuff-Type Deal,8.801235,0.611290,5.754367,21,13,1.692308,34.526201
7,Seven Thirty-Seven,8.601235,1.459071,21.428122,15,7,2.285714,64.284365
8,Grilled,9.301235,1.346462,10.663760,12,11,2.545455,53.318800
9,Bit by a Dead Bee,8.301235,0.511115,3.878651,20,17,2.823529,31.029206


In [159]:
get_corr(df_bb)

,Network Metrics,Correlation,pValue
0,weighted_density1,-0.062261,0.762539
1,weighted_density2,0.051398,0.803087
2,active_nodes,-0.138513,0.499797
3,agg_actnodes,-0.278249,0.168705
4,avg_degrees,-0.030753,0.881448
5,avg_strength,0.160387,0.433815


# GETTING REVIEWS FROM IMDB

House of Cards


,Episode,Review
0,Chapter 1,8.601235
1,Chapter 2,8.501235
2,Chapter 3,8.301235
3,Chapter 4,8.201235
4,Chapter 5,8.401235
5,Chapter 6,8.501235
6,Chapter 7,8.101235
7,Chapter 8,7.701235
8,Chapter 9,8.501235
9,Chapter 10,8.701235


In [348]:
HoC_review = get_review("1856010", 3, 14)
HoC_review.to_csv("HoC.csv")
GoT_review = get_review("0944947", 4, 11)
GoT_review.to_csv("GoT.csv")
BB_review = get_review("0903747", 4, 14)
BB_review.to_csv("BB.csv")


House of Cards
Game of Thrones
Breaking Bad


In [349]:
HoC_review = pd.read_csv('BB.csv')  
HoC_review.drop('Unnamed: 0', 1)

,Episode,Review
0,Pilot,9.001235
1,Cat's in the Bag...,8.601235
2,...And the Bag's in the River,8.701235
3,Cancer Man,8.201235
4,Gray Matter,8.301235
5,Crazy Handful of Nothin',9.301235
6,A No-Rough-Stuff-Type Deal,8.801235
7,Seven Thirty-Seven,8.601235
8,Grilled,9.301235
9,Bit by a Dead Bee,8.301235


In [350]:
BB_review = get_review("0903747", 4, 14)
BB_review

Breaking Bad


,Episode,Review
0,Pilot,9.001235
1,Cat's in the Bag...,8.601235
2,...And the Bag's in the River,8.701235
3,Cancer Man,8.201235
4,Gray Matter,8.301235
5,Crazy Handful of Nothin',9.301235
6,A No-Rough-Stuff-Type Deal,8.801235
7,Seven Thirty-Seven,8.601235
8,Grilled,9.301235
9,Bit by a Dead Bee,8.301235


In [ ]:
import imdb

# creating instance of IMDb
ia = imdb.IMDb()
 
def get_review(code, num_seasons, num_epsodes):
    # id
    #code = "1856010"

    # getting information
    series = ia.get_movie(code)

    # adding new info set
    ia.update(series, 'episodes')

    # getting episodes of the series
    episodes = series.data['episodes']

    # printing the object i.e name
    print(series)

    dict = {'Episode':[],
            'Review':[]
           }

    # getting season
    for s in range(1,num_seasons):
        season = episodes[s]

        # getting single episode of season
        for i in range(1,num_epsodes):
            try:
                epi = season[i]

                # getting id and printing it
                #print(epi['title'], epi.data['rating'])
                dict['Episode'].append(epi['title'])
                dict['Review'].append(epi.data['rating'])
            except:
                pass

    df_review = pd.DataFrame(dict)
    return df_review

HoC_review = get_review("1856010", 3, 14)
HoC_review